In [5]:
#TO GET ALL THE GAMES IN A SEASON
from nba_api.stats.endpoints import leaguegamefinder
from sympy.codegen.ast import continue_

#Get regular season games
gamefinder = leaguegamefinder.LeagueGameFinder(
    season_nullable='2025-26',
    league_id_nullable='00',
    season_type_nullable='Regular Season'
)
games = gamefinder.get_data_frames()[0]
game_ids = games['GAME_ID'].unique().tolist()
len(game_ids)

611

In [13]:
game_ids

['0022500583',
 '0022500582',
 '0022500585',
 '0022500586',
 '0022500581',
 '0022500580',
 '0022500579',
 '0022500584',
 '0022500578',
 '0022500576',
 '0022500577',
 '0022500574',
 '0022500572',
 '0022500575',
 '0022500573',
 '0022500571',
 '0022500565',
 '0022500566',
 '0022500567',
 '0022500569',
 '0022500570',
 '0022500568',
 '0022500564',
 '0022500559',
 '0022500558',
 '0022500561',
 '0022500563',
 '0022500560',
 '0022500562',
 '0022500553',
 '0022500557',
 '0022500556',
 '0022500548',
 '0022500551',
 '0022500550',
 '0022500549',
 '0022500552',
 '0022500555',
 '0022500554',
 '0022500544',
 '0022500546',
 '0022500542',
 '0022500543',
 '0022500547',
 '0022500545',
 '0022500532',
 '0022500534',
 '0022500540',
 '0022500541',
 '0022500537',
 '0022500535',
 '0022500533',
 '0022500539',
 '0022500538',
 '0022500536',
 '0022500530',
 '0022500528',
 '0022500531',
 '0022500524',
 '0022500523',
 '0022500518',
 '0022500521',
 '0022500525',
 '0022500516',
 '0022500519',
 '0022500522',
 '00225005

In [36]:
from nba_api.stats.endpoints import BoxScoreTraditionalV3
from nba_api.stats.endpoints import playbyplayv3
from nba_api.stats.endpoints import leaguegamefinder
import time
import random
from requests.exceptions import ReadTimeout, ConnectionError, RequestException, Timeout
from json.decoder import JSONDecodeError
import re
import pandas as pd
def is_int_convertible(x):
    try:
        int(x)
        return True
    except (ValueError, TypeError):
        return False

def get_minutes_from_v3_clock(clock_str):
    if not isinstance(clock_str, str): return 0

    # Check for PTxxM format
    match = re.search(r'PT(\d+)M', clock_str)
    if match:
        return int(match.group(1))

    # Check for MM:SS format
    if ':' in clock_str:
        return int(clock_str.split(':')[0])

    return 0


gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2025-26',
                                              league_id_nullable='00',
                                              season_type_nullable='Regular Season')
games = gamefinder.get_data_frames()[0]
pbp = playbyplayv3.PlayByPlayV3("0022500544").get_data_frames()[0]
stats_df = BoxScoreTraditionalV3(game_id="0022500544").get_data_frames()[0]

game_dict = {}
game="0022500544"

try:
    game_dict['GAME_ID']=game
    game_dict['TEAM1']= str(games[games["GAME_ID"]== game].iloc[0]["TEAM_ABBREVIATION"])
    game_dict['TEAM2']= str(games[games["GAME_ID"]== game].iloc[1]["TEAM_ABBREVIATION"])
    game_dict['DATE']= str(games[games["GAME_ID"]== game].iloc[0]["GAME_DATE"])
    game_dict['PF']= games[games["GAME_ID"]== game].iloc[0]["PF"] + games[games["GAME_ID"]== game].iloc[1]["PF"]

    # --- A. Clutch Calculation ---
    period = pbp['period'].unique().max()
    if period == 5:
        game_dict['CLUTCH'] = 0.95
    elif period >= 6:
        game_dict['CLUTCH'] = 1.0
    else:
#           Filter for 4th quarter +
        pbp_valid = pbp[pbp['period'] >= 4].copy()

        # Filter for last 5 minutes using helper (We apply the function to the 'clock' column)
        pbp_valid['MINUTES_REMAINING'] = pbp_valid['clock'].apply(get_minutes_from_v3_clock)
        pbp_valid = pbp_valid[pbp_valid['MINUTES_REMAINING'] < 5]

        # Calculate Margin Manually (Home - Away), Ensure we treat None/NaN as 0 or forward fill if needed
        pbp_valid['scoreHome'] = pd.to_numeric(pbp_valid['scoreHome'], errors='coerce')
        pbp_valid['scoreAway'] = pd.to_numeric(pbp_valid['scoreAway'], errors='coerce')

        pbp_valid[['scoreHome', 'scoreAway']] = pbp_valid[['scoreHome', 'scoreAway']].ffill()
        pbp_valid = pbp_valid.dropna(subset=['scoreHome', 'scoreAway'])

        pbp_valid['margin'] = pbp_valid['scoreHome'] - pbp_valid['scoreAway']


        # Count possessions within +/- 5 points
        clutch_mask = (pbp_valid['margin'] <= 5) & (pbp_valid['margin'] >= -5)
        clutch_count = clutch_mask.sum()

        game_dict['CLUTCH'] = clutch_count

    # --- B. Performance Calculation ---
    if stats_df is None:
        print(f"  [x] Failed to fetch BoxScore for {game}. Skipping.")


    stats_df['GAME_SCORE'] = (
        stats_df['points'] + 0.4 * stats_df['fieldGoalsMade'] - 0.7 * stats_df['fieldGoalsAttempted'] -
        0.4 * (stats_df['freeThrowsAttempted'] - stats_df['freeThrowsMade']) + 0.7 * stats_df['reboundsOffensive'] +
        0.3 * stats_df['reboundsDefensive'] + stats_df['steals'] + 0.7 * stats_df['assists'] +
        0.7 * stats_df['blocks'] - 0.4 * stats_df['foulsPersonal'] - stats_df['turnovers']
    )

    performance_sum = 0.0
    for gs in stats_df['GAME_SCORE']:
        if gs >= 70.0: performance_sum += 0.95
        elif gs >= 60.0: performance_sum += 0.9
        elif gs >= 50.0: performance_sum += 0.8
        elif gs >= 40.0: performance_sum += 0.7

    game_dict['PERFORMANCE'] = performance_sum


    # --- C. Highlights Calculation ---
    pbp['description'] = pbp['description'].astype(str).str.upper()

    # Dunks: look for 'DUNK' in the single description column
    # Filter out 'MISS' to be safe, though usually 'Dunk' implies made in text
    if 'shotResult' in pbp.columns:
        dunks = pbp[
            (pbp['description'].str.contains("DUNK", case=False)) &
            (pbp['shotResult'] == 'Made')
        ].shape[0]
    else:
        # Fallback if specific column is missing
        dunks = pbp[
            (pbp['description'].str.contains("DUNK", case=False)) &
            (~pbp['description'].str.contains("MISS", case=False))
        ].shape[0]

    # Fix for threes: ensure we grab values safely
    threes= int(games[games["GAME_ID"]== game].iloc[0]["FG3M"] + games[games["GAME_ID"]== game].iloc[1]["FG3M"])

    blocks = pbp[pbp['description'].str.contains("BLOCK")].shape[0]

    #calculate highlights/minutes
    total_highlights = dunks + (0.75 * threes) + blocks
    highlights_density = total_highlights / 48

    game_dict['HIGHLIGHTS'] = highlights_density

    # Standard sleep
    time.sleep(random.uniform(0.5, 1.0))

except Exception as e:
    print(f"  [!!!] Critical Error processing game {game}: {e}")

print(clutch_mask)
stats_df["GAME_SCORE"]
print(dunks)
print(threes)
print(blocks)


473    False
474     True
481     True
486     True
491     True
500     True
503     True
504     True
505     True
509     True
515     True
517     True
519     True
540     True
554     True
555    False
559    False
Name: margin, dtype: bool
12
19
19


In [47]:
#How to get play by play data
from nba_api.stats.endpoints import playbyplayv3
pbp = playbyplayv3.PlayByPlayV3("0022500332")
pbp = pbp.get_data_frames()[0]
#
# player_stat_data = BoxScoreTraditionalV2(game_id='0022401161')
# stats_df = player_stat_data.player_stats.get_data_frame()
pbp

,gameId,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,...,scoreHome,scoreAway,pointsTotal,location,description,actionType,subType,videoAvailable,shotValue,actionId
0,0022500332,2,PT12M00.00S,1,0,,0,,,0,...,0,0,0,,Start of 1st Period (8:41 PM EST),period,start,1,0,1
1,0022500332,7,PT12M00.00S,1,1610612742,DAL,203076,Davis,A. Davis,0,...,,,0,h,Jump Ball Davis vs. Adebayo: Tip to Herro,Jump Ball,,1,0,2
2,0022500332,10,PT11M43.00S,1,1610612748,MIA,1628389,Adebayo,B. Adebayo,5,...,0,3,3,v,Adebayo 26' 3PT Jump Shot (3 PTS) (Wiggins 1 AST),Made Shot,Jump Shot,1,3,3
3,0022500332,12,PT11M16.00S,1,1610612742,DAL,1642948,Nembhard,R. Nembhard,-132,...,,,0,h,MISS Nembhard 13' Step Back Jump Shot,Missed Shot,Step Back Jump shot,1,2,4
4,0022500332,13,PT11M13.00S,1,1610612748,MIA,203952,Wiggins,A. Wiggins,0,...,,,0,v,Wiggins REBOUND (Off:0 Def:1),Rebound,Unknown,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,0022500332,673,PT00M24.20S,4,1610612742,DAL,1642948,Nembhard,R. Nembhard,-67,...,,,0,h,MISS Nembhard 21' Step Back Jump Shot,Missed Shot,Step Back Jump shot,1,2,484
484,0022500332,674,PT00M20.80S,4,1610612748,MIA,1628389,Adebayo,B. Adebayo,0,...,,,0,v,Adebayo REBOUND (Off:1 Def:6),Rebound,Unknown,1,0,485
485,0022500332,675,PT00M08.80S,4,1610612748,MIA,1628389,Adebayo,B. Adebayo,-195,...,,,0,v,MISS Adebayo 27' 3PT Pullup Jump Shot,Missed Shot,Pullup Jump shot,1,3,486
486,0022500332,676,PT00M05.20S,4,1610612742,DAL,1642948,Nembhard,R. Nembhard,0,...,,,0,h,Nembhard REBOUND (Off:0 Def:5),Rebound,Unknown,1,0,487


In [ ]:
##.............Game Data Ingestion Loop.........##
maxClutch=0
maxHighlight=0
maxPerformance=0
for game in game_ids:#[939:1230]:
    dict={}

#Get game id
    dict['GAME_ID']=game

#Get teams, date, fouls
    dict['TEAM1']= str(games[games["GAME_ID"]== game].iloc[0]["TEAM_ABBREVIATION"])
    dict['TEAM2']= str(games[games["GAME_ID"]== game].iloc[1]["TEAM_ABBREVIATION"])
    dict['DATE']= str(games[games["GAME_ID"]== game].iloc[0]["GAME_DATE"])
    dict['PF']= games[games["GAME_ID"]== game].iloc[0]["PF"] + games[games["GAME_ID"]== game].iloc[1]["PF"]

#Extract PBP data
    pbp = playbyplayv2.PlayByPlayV2(game)
    pbp = pbp.get_data_frames()[0]

#Clutch
    period= pbp['PERIOD'].unique().max()
    if period==5:
        dict['CLUTCH']= 0.95
    elif period>=6:
        dict['CLUTCH']= 1.0
    else:
        #filtering plays that happened in 4th quarter or later, with 5 minutes or less left, with Score margin is not missing, and where score margin can be convertable to an integer
        pbp_valid= pbp[(pbp["PERIOD"]>=4) & (pbp["PCTIMESTRING"].str.split(":").str[0].astype(int) < 5) & (pbp['SCOREMARGIN'].notna()) & (pbp['SCOREMARGIN'].apply(is_int_convertible))]

        #count the number of clutch possessions
        clutch=(pbp_valid["SCOREMARGIN"].astype(int) <= 5) & (pbp_valid["SCOREMARGIN"].astype(int) >= -5)
        clutch_count= (clutch==True).sum()

        dict['CLUTCH']= clutch_count
        if clutch_count>maxClutch:
             maxClutch= clutch_count

#Performance
    #Extract boxscore data
    player_stat_data = BoxScoreTraditionalV2(game)
    stats_df = player_stat_data.player_stats.get_data_frame()

    stats_df['GAME_SCORE'] = (
            stats_df['PTS'] + 0.4 * stats_df['FGM'] - 0.7 * stats_df['FGA'] -
            0.4 * (stats_df['FTA'] - stats_df['FTM']) + 0.7 * stats_df['OREB'] +
            0.3 * stats_df['DREB'] + stats_df['STL'] + 0.7 * stats_df['AST'] +
            0.7 * stats_df['BLK'] - 0.4 * stats_df['PF'] - stats_df['TOV']
        )

    performance_sum=0.0
    for gs in stats_df['GAME_SCORE']:
        if gs>=70.0:
            performance_sum+=0.95
        elif gs>=60.0:
            performance_sum+=0.9
        elif gs>=50.0:
            performance_sum+=0.8
        elif gs>=40.0:
            performance_sum+=0.7

    dict['PERFORMANCE']= performance_sum
    if performance_sum>maxPerformance:
         maxPerformance= performance_sum


#Highlights
    dunks= pbp[pbp["HOMEDESCRIPTION"].str.contains("Dunk", case=False, na=False) & ~pbp["HOMEDESCRIPTION"].str.contains("MISS", case=False, na=False) |pbp["VISITORDESCRIPTION"].str.contains("Dunk", case=False, na=False) & ~pbp["VISITORDESCRIPTION"].str.contains("MISS", case=False, na=False) ].shape[0]

    threes= games[games["GAME_ID"]== game].iloc[0]["FG3M"] + games[games["GAME_ID"]== game].iloc[1]["FG3M"]

    blocks= pbp[pbp["HOMEDESCRIPTION"].str.contains("BLOCK", case=False, na=False) | pbp["VISITORDESCRIPTION"].str.contains("BLOCK", case=False, na=False)].shape[0]

    total_highlights = len(dunks) + 0.75*len(threes) + len(blocks)
    highlights_density = total_highlights / 48 #highlights per min
    dict['HIGHLIGHTS']= highlights_density
    if highlights_density>maxHighlight:
         maxHighlight= highlights_density


    df_list.append(dict)

            # Respectful delay to avoid rate limiting
    time.sleep(random.uniform(0.8, 1.5))
